In [18]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from adjustText import adjust_text
import numpy as np

import nfl_data_py as nfl

In [19]:
# Turn off max columns for pandas DataFrame
pd.set_option('display.max_columns', None)

In [20]:
years = range(2017, 2023)

In [21]:
data = nfl.import_pbp_data(years, cache=True, alt_path="../cache")

2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.


In [154]:
df = data[['game_id', 'play_id', 'season', 'posteam', 'play_type', 'season_type', 'offense_players', 'home_coach', 'away_coach', 'home_team', 'away_team']]

In [155]:
# Filter to only where play_type is run or pass
df = df[df['play_type'].isin(['run', 'pass'])]

# Filter to only where season_type is REG
df = df[df['season_type'] == 'REG']

In [156]:
ids = nfl.import_ids()

In [157]:
ids = ids[['gsis_id', 'position', 'name']]
ids = ids.dropna()

are you accounbting for differnt order of same players?
also should I maybe add a new column for each player and that way I can sum up as I please and get hey this is the coach's top combo? idk

unique play_id??

In [159]:
# First, we need to create two new columns for the 'coach' and 'team'.
# This step is required because the ball can be with either home team or away team.
df['coach'] = df.apply(lambda row: row['home_coach'] if row['posteam']
                       == row['home_team'] else row['away_coach'], axis=1)


In [ ]:
# Unpack the 'offense_players' field into separate rows
df = df['offense_players'].str.split(';', expand=True)\
    .stack()\
    .reset_index(level=-1, drop=True)\
    .rename('gsis_id')\
    .reset_index()\
    .merge(df.drop(columns='offense_players'), left_on='index', right_index=True)


In [ ]:
# Join with 'ids' dataframe
df = df.merge(ids, on='gsis_id')


In [ ]:
# Filter for 'WR', 'TE', 'RB' positions
df = df[df['position'].isin(['WR', 'TE', 'RB'])]

In [ ]:
# Group by game_id and play_id to get sets of players for each play
grouped = df.groupby(['game_id', 'play_id'])[
    'gsis_id'].apply(set).reset_index()

In [ ]:
# Rename the new column
grouped.rename(columns={'gsis_id': 'players'}, inplace=True)

In [ ]:
# Merge this new dataframe with the original one
df = pd.merge(df, grouped, on=['game_id', 'play_id'], how='left')


In [ ]:
# Drop the duplicates
df = df.drop_duplicates(['game_id', 'play_id'])


In [ ]:
# Group by season, coach and the players set, then reset index
unique_combinations_df = df.groupby(
    ['season', 'coach', 'players']).size().reset_index()


In [ ]:
# Now we just need to count unique rows for each season and coach
unique_combinations_count = unique_combinations_df.groupby(
    ['season', 'coach']).size().reset_index(name='num_unique_combinations')

In [ ]:
# Merge the unique combinations count back into the original dataframe
df = pd.merge(df, unique_combinations_count, on=[
              'season', 'coach'], how='left')
